In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 10 #10:20%, 20: 40%, 30:60%
train_image_s2 = 20 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = ((image_no<train_image)&(ALL_participant_class!=2))|((image_no<train_image_s2)&(ALL_participant_class==2))
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
# sum(Train_participant_class)


## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

## Speaker 6 Test dataset 

In [15]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [16]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [17]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [18]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [19]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [20]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [21]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [22]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [23]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [24]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
#                                       'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
#                                       'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_training_Data_Size.csv')
# Perfomance
Perfomance                                                        

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,20%,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,20%,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,20%,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,20%,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,20%,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
5,Group,20%,44.229166,0.9705,0.9521,1.000,0.9550,0.9074,0.8796,0.9826,0.9739,0.9633,0.9633,1.00,0.99,0.62,0.456,0.3069
6,Group,20%,44.535541,0.9632,0.9540,0.991,0.9369,0.9259,0.8981,0.9565,0.9652,0.9541,0.9817,0.99,0.99,0.67,0.496,0.3267
7,Group,20%,42.948731,0.9724,0.9595,1.000,0.9550,0.9074,0.8889,0.9913,0.9739,0.9633,0.9908,1.00,0.99,0.67,0.496,0.2574
8,Group,20%,45.444911,0.9724,0.9503,0.991,0.9459,0.9167,0.8796,0.9913,0.9652,0.9633,0.9725,1.00,0.99,0.60,0.552,0.2673
9,Group,20%,42.707019,0.9669,0.9540,0.991,0.9459,0.9167,0.8704,0.9739,0.9652,0.9541,0.9908,1.00,1.00,0.66,0.480,0.4059


# Create model 

In [25]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [26]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs, 
                           {resnet_model.layers[-2].name:Train_participant_class, 
                            resnet_model.layers[-1].name:Train_command_class,
                            resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                            resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {resnet_model.layers[-2].name:Val_participant_class,
                                              resnet_model.layers[-1].name:Val_command_class,
                                              resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                              resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Group",'Size':'mix_20%&s2_40%','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        resnet_model.save('Initial_group_model_mix_20p_40p_0608.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('mix_20p_40p_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('mix_20p_40p_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
10/10 [==============================] - 4s 416ms/step - loss: 3.7580 - participant_output_loss: 1.5107 - command_output_loss: 2.2426 - command_output_1_loss: 7.2169e-04 - participant_output_1_loss: 0.0040 - participant_output_accuracy: 0.3850 - command_output_accuracy: 0.2717 - command_output_1_accuracy: 0.0033 - participant_output_1_accuracy: 0.1500 - val_loss: 3.3565 - val_participant_output_loss: 1.3232 - val_command_output_loss: 2.0329 - val_command_output_1_loss: 1.5829e-04 - val_participant_output_1_loss: 1.8299e-04 - val_participant_output_accuracy: 0.5028 - val_command_output_accuracy: 0.4567 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2302
Epoch 2/10
10/10 [==============================] - 2s 189ms/step - loss: 3.0626 - participant_output_loss: 1.1257 - command_output_loss: 1.9367 - command_output_1_loss: 9.6669e-05 - participant_output_1_loss: 1.2350e-04 - participant_output_accuracy: 0.6417 - command_output_accuracy: 0.6217

Epoch 3/10
10/10 [==============================] - 1s 146ms/step - loss: 1.0215 - participant_output_loss: 0.1523 - command_output_loss: 0.8691 - command_output_1_loss: 4.5990e-05 - participant_output_1_loss: 1.9762e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9967 - command_output_1_accuracy: 0.1250 - participant_output_1_accuracy: 0.2133 - val_loss: 1.1791 - val_participant_output_loss: 0.2515 - val_command_output_loss: 0.9275 - val_command_output_1_loss: 3.7399e-05 - val_participant_output_1_loss: 2.4843e-05 - val_participant_output_accuracy: 0.9613 - val_command_output_accuracy: 0.9576 - val_command_output_1_accuracy: 0.1123 - val_participant_output_1_accuracy: 0.2192
Epoch 4/10
10/10 [==============================] - 2s 193ms/step - loss: 0.9326 - participant_output_loss: 0.1346 - command_output_loss: 0.7979 - command_output_1_loss: 3.7155e-05 - participant_output_1_loss: 1.8767e-05 - participant_output_accuracy: 0.9983 - command_output_accuracy: 0.9967

Epoch 5/10
10/10 [==============================] - 2s 221ms/step - loss: 2.2568 - participant_output_loss: 0.5512 - command_output_loss: 1.7055 - command_output_1_loss: 3.5627e-05 - participant_output_1_loss: 4.0271e-05 - participant_output_accuracy: 0.9300 - command_output_accuracy: 0.9033 - command_output_1_accuracy: 0.4717 - participant_output_1_accuracy: 0.2283 - val_loss: 2.2963 - val_participant_output_loss: 0.6009 - val_command_output_loss: 1.6953 - val_command_output_1_loss: 6.1481e-05 - val_participant_output_1_loss: 3.8558e-05 - val_participant_output_accuracy: 0.9337 - val_command_output_accuracy: 0.8729 - val_command_output_1_accuracy: 0.5083 - val_participant_output_1_accuracy: 0.3315
Epoch 6/10
10/10 [==============================] - 2s 217ms/step - loss: 2.0815 - participant_output_loss: 0.4597 - command_output_loss: 1.6217 - command_output_1_loss: 9.5810e-05 - participant_output_1_loss: 3.1110e-05 - participant_output_accuracy: 0.9550 - command_output_accuracy: 0.9467

Epoch 7/10
10/10 [==============================] - 2s 213ms/step - loss: 0.9067 - participant_output_loss: 0.0912 - command_output_loss: 0.8154 - command_output_1_loss: 6.7052e-05 - participant_output_1_loss: 1.5814e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9983 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2083 - val_loss: 1.0787 - val_participant_output_loss: 0.1868 - val_command_output_loss: 0.8918 - val_command_output_1_loss: 6.0846e-05 - val_participant_output_1_loss: 2.5813e-05 - val_participant_output_accuracy: 0.9687 - val_command_output_accuracy: 0.9650 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1418
Epoch 8/10
10/10 [==============================] - 2s 216ms/step - loss: 0.8413 - participant_output_loss: 0.0828 - command_output_loss: 0.7584 - command_output_1_loss: 3.8235e-05 - participant_output_1_loss: 1.5057e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy

Epoch 9/10
10/10 [==============================] - 2s 212ms/step - loss: 1.4873 - participant_output_loss: 0.2100 - command_output_loss: 1.2772 - command_output_1_loss: 4.7651e-05 - participant_output_1_loss: 3.8226e-05 - participant_output_accuracy: 0.9933 - command_output_accuracy: 0.9833 - command_output_1_accuracy: 0.0033 - participant_output_1_accuracy: 0.2083 - val_loss: 1.5802 - val_participant_output_loss: 0.2901 - val_command_output_loss: 1.2900 - val_command_output_1_loss: 4.8229e-05 - val_participant_output_1_loss: 4.6182e-05 - val_participant_output_accuracy: 0.9705 - val_command_output_accuracy: 0.9411 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1952
Epoch 10/10
10/10 [==============================] - 2s 211ms/step - loss: 1.3589 - participant_output_loss: 0.1753 - command_output_loss: 1.1835 - command_output_1_loss: 6.3262e-05 - participant_output_1_loss: 3.6822e-05 - participant_output_accuracy: 0.9950 - command_output_accuracy: 0

...

Epoch 6/10
10/10 [==============================] - 2s 198ms/step - loss: 1.9540 - participant_output_loss: 0.5047 - command_output_loss: 1.4491 - command_output_1_loss: 6.8819e-05 - participant_output_1_loss: 4.3230e-05 - participant_output_accuracy: 0.9650 - command_output_accuracy: 0.9417 - command_output_1_accuracy: 0.0033 - participant_output_1_accuracy: 0.2033 - val_loss: 2.0207 - val_participant_output_loss: 0.5781 - val_command_output_loss: 1.4425 - val_command_output_1_loss: 4.8650e-05 - val_participant_output_1_loss: 4.5903e-05 - val_participant_output_accuracy: 0.9392 - val_command_output_accuracy: 0.9134 - val_command_output_1_accuracy: 0.0368 - val_participant_output_1_accuracy: 0.1934
Epoch 7/10
10/10 [==============================] - 2s 194ms/step - loss: 1.8045 - participant_output_loss: 0.4545 - command_output_loss: 1.3499 - command_output_1_loss: 4.1949e-05 - participant_output_1_loss: 4.0141e-05 - participant_output_accuracy: 0.9750 - command_output_accuracy: 0.9683

Epoch 8/10
10/10 [==============================] - 1s 144ms/step - loss: 0.6542 - participant_output_loss: 0.1460 - command_output_loss: 0.5082 - command_output_1_loss: 4.9830e-05 - participant_output_1_loss: 2.5902e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0333 - participant_output_1_accuracy: 0.1633 - val_loss: 0.8530 - val_participant_output_loss: 0.2492 - val_command_output_loss: 0.6038 - val_command_output_1_loss: 3.0696e-05 - val_participant_output_1_loss: 4.0736e-05 - val_participant_output_accuracy: 0.9687 - val_command_output_accuracy: 0.9576 - val_command_output_1_accuracy: 0.0092 - val_participant_output_1_accuracy: 0.1934
Epoch 9/10
10/10 [==============================] - 2s 197ms/step - loss: 0.5949 - participant_output_loss: 0.1344 - command_output_loss: 0.4604 - command_output_1_loss: 2.8165e-05 - participant_output_1_loss: 2.5142e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 10/10
10/10 [==============================] - 2s 166ms/step - loss: 1.1324 - participant_output_loss: 0.1489 - command_output_loss: 0.9834 - command_output_1_loss: 6.4873e-05 - participant_output_1_loss: 3.6436e-05 - participant_output_accuracy: 0.9967 - command_output_accuracy: 0.9950 - command_output_1_accuracy: 0.0300 - participant_output_1_accuracy: 0.1817 - val_loss: 1.2673 - val_participant_output_loss: 0.2523 - val_command_output_loss: 1.0149 - val_command_output_1_loss: 5.7295e-05 - val_participant_output_1_loss: 4.7370e-05 - val_participant_output_accuracy: 0.9613 - val_command_output_accuracy: 0.9558 - val_command_output_1_accuracy: 0.0147 - val_participant_output_1_accuracy: 0.2173
Epoch 1/10
10/10 [==============================] - 3s 268ms/step - loss: 1.0174 - participant_output_loss: 0.1254 - command_output_loss: 0.8919 - command_output_1_loss: 5.8770e-05 - participant_output_1_loss: 3.5358e-05 - participant_output_accuracy: 0.9967 - command_output_accuracy: 0.991

Epoch 4/10
10/10 [==============================] - 2s 184ms/step - loss: 2.3154 - participant_output_loss: 0.6805 - command_output_loss: 1.6349 - command_output_1_loss: 5.6716e-05 - participant_output_1_loss: 4.6857e-05 - participant_output_accuracy: 0.8900 - command_output_accuracy: 0.8717 - command_output_1_accuracy: 0.0017 - participant_output_1_accuracy: 0.3033 - val_loss: 2.3073 - val_participant_output_loss: 0.7046 - val_command_output_loss: 1.6026 - val_command_output_1_loss: 6.6265e-05 - val_participant_output_1_loss: 4.4636e-05 - val_participant_output_accuracy: 0.8858 - val_command_output_accuracy: 0.8766 - val_command_output_1_accuracy: 0.0037 - val_participant_output_1_accuracy: 0.2376
Epoch 5/10
10/10 [==============================] - 2s 192ms/step - loss: 2.0707 - participant_output_loss: 0.5479 - command_output_loss: 1.5227 - command_output_1_loss: 7.4972e-05 - participant_output_1_loss: 3.6658e-05 - participant_output_accuracy: 0.9300 - command_output_accuracy: 0.9100

Epoch 6/10
10/10 [==============================] - 2s 191ms/step - loss: 0.6869 - participant_output_loss: 0.1215 - command_output_loss: 0.5653 - command_output_1_loss: 3.8572e-05 - participant_output_1_loss: 1.8447e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9983 - command_output_1_accuracy: 0.1400 - participant_output_1_accuracy: 0.2033 - val_loss: 0.8678 - val_participant_output_loss: 0.2266 - val_command_output_loss: 0.6412 - val_command_output_1_loss: 5.3211e-05 - val_participant_output_1_loss: 2.5913e-05 - val_participant_output_accuracy: 0.9705 - val_command_output_accuracy: 0.9687 - val_command_output_1_accuracy: 0.3094 - val_participant_output_1_accuracy: 0.1252
Epoch 7/10
10/10 [==============================] - 1s 141ms/step - loss: 0.6211 - participant_output_loss: 0.1115 - command_output_loss: 0.5095 - command_output_1_loss: 4.7410e-05 - participant_output_1_loss: 1.8056e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9983

In [27]:
resnet_model = tf.keras.models.load_model('Initial_group_model_mix_20p_40p_0608.h5')
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_c = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_s = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_c,acc_s)

0.9945 0.965


In [28]:
Perfomance

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,20%,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,20%,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,20%,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,20%,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,20%,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
5,Group,20%,44.229166,0.9705,0.9521,1.000,0.9550,0.9074,0.8796,0.9826,0.9739,0.9633,0.9633,1.00,0.99,0.62,0.456,0.3069
6,Group,20%,44.535541,0.9632,0.9540,0.991,0.9369,0.9259,0.8981,0.9565,0.9652,0.9541,0.9817,0.99,0.99,0.67,0.496,0.3267
7,Group,20%,42.948731,0.9724,0.9595,1.000,0.9550,0.9074,0.8889,0.9913,0.9739,0.9633,0.9908,1.00,0.99,0.67,0.496,0.2574
8,Group,20%,45.444911,0.9724,0.9503,0.991,0.9459,0.9167,0.8796,0.9913,0.9652,0.9633,0.9725,1.00,0.99,0.60,0.552,0.2673
9,Group,20%,42.707019,0.9669,0.9540,0.991,0.9459,0.9167,0.8704,0.9739,0.9652,0.9541,0.9908,1.00,1.00,0.66,0.480,0.4059


In [29]:
Perfomance.to_csv('Performance_0608_training_Data_Size.csv',index=False)

In [30]:
best_index

3